<a href="https://colab.research.google.com/github/nikhathina/earth-analytics-python-env/blob/main/Calculating__greenness_using_satellite_imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install earthpy
!pip install earthpy

In [ ]:
import rasterio
import numpy as np
import earthpy.plot as ep
import matplotlib.pyplot as plt
import geopandas as gpd
from matplotlib.colors import ListedColormap
from rasterio.mask import mask
from scipy import stats



band_4 = r'PATH_TO_BAND_4.TIF'
band_5= r'PATH_TO_BAND_5.TIF'

def calculate_ndvi(nir_band, red_band):
    # Calculate NDVI using the formula: (NIR - Red) / (NIR + Red)
    ndvi = (nir_band - red_band) / (nir_band + red_band)
    return ndvi

# Paths to Landsat bands (replace with your actual file paths)
nir_band_path = band_5
red_band_path = band_4

# Load NIR and Red bands using rasterio
with rasterio.open(nir_band_path) as nir_dataset, rasterio.open(red_band_path) as red_dataset:
    nir_band = nir_dataset.read(1).astype(float)
    red_band = red_dataset.read(1).astype(float)

# Calculate NDVI

ndvi = calculate_ndvi(nir_band, red_band)

print ('ndvi')

# # Optional: Save NDVI as a new raster
output_ndvi_path = r"PATH_TO_NDVI.tif"
with rasterio.open(nir_band_path) as src:
     profile = src.profile
     profile.update(dtype=rasterio.float32, count=1)
     with rasterio.open(output_ndvi_path, 'w', **profile) as dst:
       dst.write(ndvi, 1)

# #print("NDVI calculation complete.")

ep.plot_bands(ndvi,  cmap="RdYlGn", title='NDVI', vmin=-1, vmax=1)

CROP NDVI


In [ ]:

inshp = (r'PATH_TO_BOUNDING_BOX_SHAPEFILE.shp')
inRas = r"PATH_TO_NSVI.tif"
outRas = r'PATH_TO_CROPPED_NDVI.TIF'

Vector=gpd.read_file(inshp)

#Vector=Vector[Vector['HYBAS_ID']==6060122060] # Subsetting to my AOI

with rasterio.open(inRas) as src:
    Vector=Vector.to_crs(src.crs)
    # print(Vector.crs)
    out_image, out_transform=mask(src,Vector.geometry,crop=True)
    out_meta=src.meta.copy() # copy the metadata of the source DEM

out_meta.update({
    "driver":"Gtiff",
    "height":out_image.shape[1], # height starts with shape[1]
    "width":out_image.shape[2], # width starts with shape[2]
    "transform":out_transform
})


# Auto adjust subplot to fit figure size
plt.tight_layout()

with rasterio.open(outRas,'w',**out_meta) as dst:
    dst.write(out_image)

CROP CLOUD MASK

In [ ]:

inshp = (r'PATH_TO_BOUNDING_BOX_SHAPE_FILE.shp')
inRas = r'PATH_TO_CLOUD_MASK_IMAGE.tif'
outRas = r'PATH_TO_CROPPED_CLOUD_MASK_IMAGE.TIF'

Vector=gpd.read_file(inshp)

#Vector=Vector[Vector['HYBAS_ID']==6060122060] # Subsetting to my AOI

with rasterio.open(inRas) as src:
    Vector=Vector.to_crs(src.crs)
    # print(Vector.crs)
    out_image, out_transform=mask(src,Vector.geometry,crop=True)
    out_meta=src.meta.copy() # copy the metadata of the source DEM

out_meta.update({
    "driver":"Gtiff",
    "height":out_image.shape[1], # height starts with shape[1]
    "width":out_image.shape[2], # width starts with shape[2]
    "transform":out_transform
})


# Auto adjust subplot to fit figure size
plt.tight_layout()

with rasterio.open(outRas,'w',**out_meta) as dst:
    dst.write(out_image)


CLOUD percentage

In [ ]:


# Load the cloud mask raster
cloud_mask_path =r'PATH_TO_CROPPED_CLOUD_MASK_IMAGE.TIF'

with rasterio.open(cloud_mask_path) as src:
    cloud_mask = src.read(1).astype(np.uint8)

# Count cloud and non-cloud pixels
cloud_pixels = (cloud_mask == 1).sum()
total_pixels = cloud_mask.size

# Calculate percentage of cloud cover
cloud_cover_percentage = (cloud_pixels / total_pixels) * 100

print(f"Cloud Cover Percentage: {cloud_cover_percentage:.2f}%")

Normalize NDVI

In [ ]:
# Load NDVI raster, cloud mask raster, and cloud percentage value
ndvi_path = r'PATH_TO_CROPPED_NDVI.TIF'
cloud_mask_path = r'PATH_TO_CROPPED_CLOUD_MASK_IMAGE.TIF'
cloud_percentage = 0.4433  # Replace with your cloud percentage value

with rasterio.open(ndvi_path) as ndvi_src, rasterio.open(cloud_mask_path) as cloud_mask_src:
    ndvi = ndvi_src.read(1).astype(float)
    cloud_mask = cloud_mask_src.read(1).astype(bool)

# Calculate the maximum and minimum NDVI values within the non-cloud areas
min_ndvi = np.min(ndvi[~cloud_mask])
max_ndvi = np.max(ndvi[~cloud_mask])

# Normalize NDVI using cloud mask and cloud percentage
normalized_ndvi = (ndvi - cloud_percentage * min_ndvi) / (1 - cloud_percentage * min_ndvi)

# Save the normalized NDVI to a new TIFF file (optional)
output_path = r'PATH_TO_NORMALIZED_NDVI.TIF'
with rasterio.open(ndvi_path) as ndvi_src:
    profile = ndvi_src.profile
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(normalized_ndvi, 1)


PLOT_NDVI

In [ ]:
tif_path = r'PATH_TO_NORMALIZED_NDVI.TIF'
with rasterio.open(tif_path) as src:
    # Read the raster data
    normalized_ndvi = src.read(1)

    # Get the spatial transformation (georeferencing information)
    transform = src.transform

    ep.plot_bands(normalized_ndvi,  cmap="RdYlGn", title='NORMALIZED_NDVI', vmin=-1, vmax=1)

CALCULATE STATISTICS FOR NDVI

In [ ]:

def calculate_ndvi_statistics(ndvi_array):
    mean_ndvi = np.nanmean(ndvi_array)
    median_ndvi = np.nanmedian(ndvi_array)
    std_ndvi = np.nanstd(ndvi_array)
    min_ndvi = np.nanmin(ndvi_array)
    max_ndvi = np.nanmax(ndvi_array)
    return mean_ndvi, median_ndvi, std_ndvi, min_ndvi, max_ndvi

# Calculate NDVI statistics
mean_ndvi, median_ndvi, std_ndvi, min_ndvi, max_ndvi = calculate_ndvi_statistics(normalized_ndvi)

print("Mean NDVI:", mean_ndvi)
print("Median NDVI:", median_ndvi)
print("Standard Deviation NDVI:", std_ndvi)
print("Minimum NDVI:", min_ndvi)
print("Maximum NDVI:", max_ndvi)

CLASSIFY THE NDVI

In [ ]:
# Create classes and apply to NDVI results
ndvi_class_bins = [-np.inf, 0, 0.1, 0.25, 0.4, 0.52, np.inf]
ndvi_landsat_class = np.digitize(normalized_ndvi, ndvi_class_bins)

# Apply the nodata mask to the newly classified NDVI data
ndvi_landsat_class = np.ma.masked_where(
    np.ma.getmask(normalized_ndvi), ndvi_landsat_class
)
np.unique(ndvi_landsat_class)

PLOT THE CLASSIFIED NDVI

In [ ]:
# Define color map
nbr_colors = ["gray", "y", "yellowgreen", "g", "darkgreen"]
nbr_cmap = ListedColormap(nbr_colors)

# Define class names
ndvi_cat_names = [
    "No Vegetation",
    "Bare Vegetation",
    "Low Vegetation",
    "Moderate Vegetation",
    "High Vegetation",
]

# Get list of classes
classes = np.unique(ndvi_landsat_class)
classes = classes.tolist()
# The mask returns a value of none in the classes. remove that
classes = classes[0:5]

# Plot your data
fig, ax = plt.subplots(figsize=(12, 12))
im = ax.imshow(ndvi_landsat_class, cmap=nbr_cmap)

ep.draw_legend(im_ax=im, classes=classes, titles=ndvi_cat_names)
ax.set_title(
    "CLASSIFIED_ Normalized Difference Vegetation Index (NDVI)",
    fontsize=14,
)
ax.set_axis_off()

# Auto adjust subplot to fit figure size
plt.tight_layout()


# Save the plot as an image file (e.g., PNG, JPEG, etc.)
output_image_path = r'PATH_TO_CLASSIFIED_NDVI.jpeg'
plt.savefig(output_image_path, dpi=600, bbox_inches='tight')

plt.show()


DEFINE THRESHOLDS AND CALCULATE PERCENTAGE GREENNESS

In [ ]:
# Define thresholds
Bare_Vegetation_threshold= 0.1
Low_Vegetation_threshold=0.25
Moderate_Vegetation_threshold= 0.4
#High_Vegetation_threshold= 0.4

# Count pixels within each category
Bare_Vegetation_threshold_pixels = (normalized_ndvi < Bare_Vegetation_threshold).sum()
Low_Vegetation_threshold_pixels = ((normalized_ndvi >= Bare_Vegetation_threshold) & (normalized_ndvi < Low_Vegetation_threshold)).sum()
Moderate_Vegetation_threshold_pixels = ((normalized_ndvi >= Low_Vegetation_threshold) & (normalized_ndvi < Moderate_Vegetation_threshold)).sum()
High_Vegetation_threshold_pixels=(normalized_ndvi>= Moderate_Vegetation_threshold).sum()

# Calculate total number of pixels
total_pixels = normalized_ndvi.size

# Calculate percentage of greenness
percentage_Bare = (Bare_Vegetation_threshold_pixels / total_pixels) * 100
percentage_Low = (Low_Vegetation_threshold_pixels / total_pixels) * 100
percentage_Moderate_veg = (Moderate_Vegetation_threshold_pixels / total_pixels) * 100
percentage_High = (High_Vegetation_threshold_pixels/ total_pixels) * 100

print(f"Percentage of Bare Vegetation: {percentage_Bare :.2f}%")
print(f"Percentage of Low Vegetation: {percentage_Low:.2f}%")
print(f"Percentage of Moderate Vegetation: {percentage_Moderate_veg:.2f}%")
print(f"Percentage of High Vegetation: {percentage_High :.2f}%")